In [4]:
states = ["DE", "IA", "IL", "MA", "MD", "MN", "MT", "NV"]
revised_integrals_values = \
[[87.61, 87.78, 88.17, 87.99, 87.79, 87.6, 87.6, 87.9], 
[85.72, 84.84, 86.04 , 85.85, 85.41, 85.61, 85.67, 86.82],
[87.51, 86.87, 86.95 , 87.32, 87.24, 86.82, 87.09, 87.09],
[82.69, 82.08, 82.48 , 81.8, 81.26, 81.48, 81.75, 82.06],
[87.84, 87.62, 88.09 , 88.17, 87.51, 87.57, 87.78, 87.78],
[86.8,  86.79, 87.25 , 86.7, 87.2, 83.76, 87.05, 86.9],
[87.56, 87.63, 87.25 , 87.47, 87.54, 87.35, 87.48, 87.67],
[92.57, 87.25, 87.73 , 92.79, 92.76, 92.14, 91.82, 91.62]]

revised_integrals = {}
for i in range(len(states)):
    revised_integrals[states[i]] = {state: revised_integrals_values[i][j] for j, state in enumerate(states)}

# Minimize Error rather than Maximize Accuracy
for key in revised_integrals:
	for task in revised_integrals[key]:
		revised_integrals[key][task] = 100.0 - revised_integrals[key][task]

### Network Selection Algorithm

In [5]:
import math 
import numpy as np

def gen_task_combinations(tasks, rtn, index, path, path_dict):
  if index >= len(tasks):
    return 

  for i in range(index, len(tasks)):
    cur_task = tasks[i]
    new_path = path
    new_dict = {k:v for k,v in path_dict.items()}
    
    # Building from a tree with two or more tasks...
    if new_path:
      new_dict[cur_task] = 0.
      for prev_task in path_dict:
        new_dict[prev_task] += revised_integrals[prev_task][cur_task]
        new_dict[cur_task] += revised_integrals[cur_task][prev_task]
      new_path = '{}|{}'.format(new_path, cur_task)
      rtn[new_path] = new_dict
    else: # First element in a new-formed tree
      new_dict[cur_task] = 0.
      new_path = cur_task

    gen_task_combinations(tasks, rtn, i+1, new_path, new_dict)

    # Fix single-task accuracy since dfs is finished for this task.
    if '|' not in new_path:
      new_dict[cur_task] = revised_integrals[cur_task][cur_task]
      rtn[new_path] = new_dict

In [6]:
rtn = {}
tasks = list(revised_integrals.keys())
num_tasks = len(tasks)
task_combinations = gen_task_combinations(tasks=tasks, rtn=rtn, index=0, path='', path_dict={})

# Normalize by the number of times the accuracy of any given element has been summed. 
# i.e. (a,b,c) => [acc(a|b) + acc(a|c)]/2 + [acc(b|a) + acc(b|c)]/2 + [acc(c|a) + acc(c|b)]/2
for group in rtn:
  if '|' in group:
    for task in rtn[group]:
      rtn[group][task] /= (len(group.split('|')) - 1)

print(rtn)
assert(len(rtn.keys()) == 2**len(revised_integrals.keys()) - 1)
rtn_tup = [(key,val) for key,val in rtn.items()]

{'DE|IA': {'DE': 12.219999999999999, 'IA': 14.280000000000001}, 'DE|IA|IL': {'DE': 12.024999999999999, 'IA': 14.119999999999997, 'IL': 12.809999999999995}, 'DE|IA|IL|MA': {'DE': 12.020000000000001, 'IA': 14.13, 'IL': 12.766666666666666, 'MA': 17.583333333333332}, 'DE|IA|IL|MA|MD': {'DE': 12.067499999999999, 'IA': 14.245000000000001, 'IL': 12.765, 'MA': 17.8725, 'MD': 12.069999999999997}, 'DE|IA|IL|MA|MD|MN': {'DE': 12.134, 'IA': 14.274000000000001, 'IL': 12.848000000000003, 'MA': 18.002, 'MD': 12.142, 'MN': 13.051999999999998}, 'DE|IA|IL|MA|MD|MN|MT': {'DE': 12.178333333333335, 'IA': 14.283333333333333, 'IL': 12.858333333333334, 'MA': 18.043333333333333, 'MD': 12.155, 'MN': 13.034999999999998, 'MT': 12.533333333333333}, 'DE|IA|IL|MA|MD|MN|MT|NV': {'DE': 12.167142857142858, 'IA': 14.125714285714286, 'IL': 12.865714285714287, 'MA': 18.02857142857143, 'MD': 12.164285714285713, 'MN': 13.044285714285712, 'MT': 12.504285714285714, 'NV': 8.991428571428571}, 'DE|IA|IL|MA|MD|MN|NV': {'DE': 12.1

In [7]:
def select_groups(index, cur_group, best_group, best_val, splits):
  # Check if this group covers all tasks.
  task_set = set()
  for group in cur_group:
    for task in group.split('|'): task_set.add(task)
  if len(task_set) == num_tasks:
    best_tasks = {task:1e6 for task in task_set}
    
    # Compute the per-task best scores for each task and average them together.
    for group in cur_group:
      for task in cur_group[group]:
        # Minimize error.
        best_tasks[task] = min(best_tasks[task], cur_group[group][task])
    group_avg = np.mean(list(best_tasks.values()))
    
    # Compare with the best grouping seen thus far.
    if group_avg < best_val[0]:
      print(cur_group)
      best_val[0] = group_avg
      best_group.clear()
      for entry in cur_group:
        best_group[entry] = cur_group[entry]
  
  # Base case.
  if len(cur_group.keys()) == splits:
    return

  # Back to combinatorics 
  for i in range(index, len(rtn_tup)):
    selected_group, selected_dict = rtn_tup[i]

    new_group = {k:v for k,v in cur_group.items()}
    new_group[selected_group] = selected_dict

    if len(new_group.keys()) <= splits:
      select_groups(i + 1, new_group, best_group, best_val, splits)

selected_group = {}
selected_val = [100000000]
select_groups(index=0, cur_group={}, best_group=selected_group, best_val=selected_val, splits=3)
print(selected_group)
print(selected_val)

{'DE|IA': {'DE': 12.219999999999999, 'IA': 14.280000000000001}, 'DE|IA|IL': {'DE': 12.024999999999999, 'IA': 14.119999999999997, 'IL': 12.809999999999995}, 'DE|IA|IL|MA|MD|MN|MT|NV': {'DE': 12.167142857142858, 'IA': 14.125714285714286, 'IL': 12.865714285714287, 'MA': 18.02857142857143, 'MD': 12.164285714285713, 'MN': 13.044285714285712, 'MT': 12.504285714285714, 'NV': 8.991428571428571}}
{'DE|IA': {'DE': 12.219999999999999, 'IA': 14.280000000000001}, 'DE|IA|IL': {'DE': 12.024999999999999, 'IA': 14.119999999999997, 'IL': 12.809999999999995}, 'DE|IA|MA|MD|MN|MT|NV': {'DE': 12.223333333333334, 'IA': 14.153333333333336, 'MA': 18.113333333333333, 'MD': 12.206666666666665, 'MN': 13.093333333333332, 'MT': 12.463333333333333, 'NV': 8.445}}
{'DE|IA': {'DE': 12.219999999999999, 'IA': 14.280000000000001}, 'DE|IA|IL': {'DE': 12.024999999999999, 'IA': 14.119999999999997, 'IL': 12.809999999999995}, 'DE|IL|MA|MD|MN|MT|NV': {'DE': 12.158333333333333, 'IL': 12.821666666666667, 'MA': 18.046666666666663,

In [8]:
selected_group

{'DE|IL|MA': {'DE': 11.920000000000002, 'IL': 12.585, 'MA': 17.415},
 'DE|MA|MD|MN|MT|NV': {'DE': 12.224,
  'MA': 18.151999999999997,
  'MD': 12.172,
  'MN': 13.069999999999999,
  'MT': 12.482,
  'NV': 7.5840000000000005},
 'IA|NV': {'IA': 13.180000000000007, 'NV': 12.75}}